In [14]:
# Step 1: Install necessary libraries
!pip install gensim transformers torch scikit-learn tqdm

import pandas as pd
from google.colab import files
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModel
from gensim.models import Doc2Vec
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from huggingface_hub import hf_hub_download

# Upload the CSV file
uploaded = files.upload()

# Load the dataset into a DataFrame
df = pd.read_csv(next(iter(uploaded)))  # Assumes the first uploaded file is your dataset

# Map 'RequirementType' to 'labels' (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)

# Check if the 'labels' column was created correctly
print(df[['RequirementType', 'labels']].head())

# Step 5: Download and load the Doc2Vec model from Hugging Face
model_path = hf_hub_download(repo_id="RafidMehda/doc2vec_model", filename="doc2vec_model")
doc2vec_model = Doc2Vec.load(model_path)

# Extract Doc2Vec embeddings for each document in the dataset
def get_doc2vec_embeddings(index):
    doc2vec_emb = doc2vec_model.dv[str(index)]
    return torch.tensor(doc2vec_emb).numpy()

doc2vec_embeddings = [get_doc2vec_embeddings(i) for i in range(len(df))]

# Load tokenizer and model from the fine-tuned Hugging Face model
tokenizer = AutoTokenizer.from_pretrained("RafidMehda/app_review_model")
model = AutoModel.from_pretrained("RafidMehda/app_review_model")

# Function to get embeddings from the fine-tuned model with average pooling
def get_finetuned_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_embedding = torch.mean(last_hidden_state, dim=1)  # Average pooling
    return pooled_embedding.squeeze().numpy()

# Generate embeddings using the fine-tuned model for the dataset
finetuned_embeddings = [get_finetuned_embeddings(doc) for doc in df['content']]

# Combine Doc2Vec and fine-tuned model embeddings
combined_embeddings = [np.concatenate((doc2vec_emb, finetuned_emb)) for doc2vec_emb, finetuned_emb in zip(doc2vec_embeddings, finetuned_embeddings)]

# Convert to numpy arrays for input
X = np.array(combined_embeddings)
y = df['labels'].values

# Use PCA to reduce the dimensionality (if needed)
pca = PCA(n_components=200)  # Reduce to 200 dimensions
X_reduced = pca.fit_transform(X)

# Introduce some label noise by shuffling 3% of labels randomly (optional step)
np.random.seed(42)
noise_ratio = 0.03  # 3% noise
num_noisy_labels = int(noise_ratio * len(y))
noisy_indices = np.random.choice(len(y), num_noisy_labels, replace=False)

# Flip the labels at noisy indices
y[noisy_indices] = 1 - y[noisy_indices]  # Invert the labels

# Convert the reduced embeddings to torch tensors
X_tensor = torch.tensor(X_reduced).float()
y_tensor = torch.tensor(y).long()

# Define a PyTorch dataset and dataloader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Increase batch size to 32

# Define a neural network classifier with reduced units and increased dropout
class CombinedEmbeddingClassifier(nn.Module):
    def __init__(self, input_dim, num_labels):
        super(CombinedEmbeddingClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Reduced from 512 to 256
        self.dropout1 = nn.Dropout(0.5)  # Increased dropout to 0.5
        self.fc2 = nn.Linear(256, 128)  # Reduced from 256 to 128
        self.dropout2 = nn.Dropout(0.5)  # Increased dropout to 0.5
        self.fc3 = nn.Linear(128, num_labels)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Instantiate the classifier model
input_dim = X_reduced.shape[1]  # The size of the combined embeddings (after PCA)
num_labels = 2  # We have two labels: Functional and Non-Functional
model = CombinedEmbeddingClassifier(input_dim=input_dim, num_labels=num_labels)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)  # AdamW optimizer with weight decay
loss_fn = nn.CrossEntropyLoss()

# Training the model
def train_model(epochs=3):  # Reduced epochs to 3 instead of 5
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch + 1}/{epochs} | Loss: {total_loss/len(dataloader):.4f}')

# Train the classifier
train_model(epochs=3)  # Train for 3 epochs instead of 5

# Cross-validation with KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_accuracies = []

# Cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_reduced)):
    print(f"Fold {fold + 1}/10")

    # Split into training and validation sets
    X_train, X_val = X_reduced[train_index], X_reduced[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Convert to torch tensors
    train_dataset = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
    val_dataset = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Batch size increased to 32
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Training the classifier
    model.train()
    for epoch in range(3):  # Train for 3 epochs
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

    # Validation
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)

    # Evaluate fold accuracy
    fold_accuracy = accuracy_score(y_val, all_preds)
    fold_accuracies.append(fold_accuracy)
    print(f"Fold {fold + 1} Accuracy: {fold_accuracy * 100:.2f}%")

# Calculate and print average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f"\nAverage Validation Accuracy across 10 folds: {average_accuracy * 100:.2f}%")


Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews (12).csv
  RequirementType  labels
0               F       1
1              NF       0
2               F       1
3              NF       0
4              NF       0


100%|██████████| 391/391 [00:01<00:00, 323.46it/s]


Epoch 1/3 | Loss: 0.2778


100%|██████████| 391/391 [00:01<00:00, 291.26it/s]


Epoch 2/3 | Loss: 0.1453


100%|██████████| 391/391 [00:01<00:00, 230.59it/s]


Epoch 3/3 | Loss: 0.1437
Fold 1/10
Fold 1 Accuracy: 69.92%
Fold 2/10
Fold 2 Accuracy: 99.52%
Fold 3/10
Fold 3 Accuracy: 100.00%
Fold 4/10
Fold 4 Accuracy: 99.84%
Fold 5/10
Fold 5 Accuracy: 100.00%
Fold 6/10
Fold 6 Accuracy: 99.92%
Fold 7/10
Fold 7 Accuracy: 99.92%
Fold 8/10
Fold 8 Accuracy: 99.92%
Fold 9/10
Fold 9 Accuracy: 100.00%
Fold 10/10
Fold 10 Accuracy: 100.00%

Average Validation Accuracy across 10 folds: 96.90%
